In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re
import undetected_chromedriver as uc
import time

# Login Bibliopass

Bibliopass is a remote publisher library owned by the University of Turin. Each remote publisher library may have a different login interface/scheme so you need to modify this login code. If you want to use a ScienceDirect subscription account, you also need to modify this login code. If you do not have access to log in to ScienceDirect, you cannot use this code (when we are not logged in, we may able to use this code but ScienceDirect usually only gives us a few pages of the search result so that the search result is not complete).

In [2]:
def login_sciencedirect():
    sciencedirect_url = "https://www-sciencedirect-com.bibliopass.unito.it/"
    driver = uc.Chrome()
    driver.get(sciencedirect_url)
    time.sleep(5)
    try:
        username = driver.find_element_by_id("username")
        password = driver.find_element_by_id("password")
        username.send_keys("your_username")
        password.send_keys("your_password")
        driver.find_element_by_name("_eventId_proceed").click()
    except:
        pass
    time.sleep(5)
    return driver

# Get All Paper Metadata

In [3]:
sciencedirect = pd.read_excel("./paper-links/PaperLinks_ScienceDirect.xlsx")

In [ ]:
# Get the paper metadata
list_source, list_querystring, list_querylinks, list_paperlinks = [], [], [], []
list_year, list_title, list_conf_jour, list_authors, list_keywords, list_abstract = [], [], [], [], [], []
driver = uc.Chrome() # driver = login_sciencedirect() # change not using bibliopass because it blocked ._.
for index in range(0,len(sciencedirect["Paper Links"])-1):
    try:
        # Get Page, change not using bibliopass because it blocked ._.
        paper_url = re.sub("www-sciencedirect-com.bibliopass.unito.it","www.sciencedirect.com",sciencedirect["Paper Links"][index])
        driver.get(paper_url)
        time.sleep(5)
        # Get Year
        try:
            year_html = driver.find_element_by_xpath("//meta[@name='citation_publication_date']")
            year = year_html.get_attribute("content").split("/")[0].strip()
        except:
            year = "failed_to_scrap"
        # Get Title
        try:
            title_html = driver.find_element_by_xpath("//meta[@name='citation_title']")
            title = title_html.get_attribute("content").strip()
        except:
            title = "failed_to_scrap"
        # Get Conf/Journal Name
        try:
            conf_jour_html = driver.find_element_by_xpath("//meta[@name='citation_journal_title']")
            conf_jour_name = conf_jour_html.get_attribute("content").strip()
        except:
            conf_jour_name = "failed_to_scrap"
        # Get Authors
        try:
            get_authors = driver.find_elements_by_css_selector("a[class='author size-m workspace-trigger']")
            for i in range(0,len(get_authors)):
                if i == 0:
                    authors = get_authors[i].text.split("\n")[0]
                else:
                    authors = authors+", "+get_authors[i].text.split("\n")[0]
        except:
            authors = "failed_to_scrap" 
        # Get Keywords
        try:
            get_keywords = driver.find_elements_by_css_selector("div[class='keyword']")
            for i in range(0,len(get_keywords)):
                if i == 0:
                    keywords = get_keywords[i].text.split("\n")[0]
                else:
                    keywords = keywords+", "+get_keywords[i].text.split("\n")[0]
        except:
            keywords = "failed_to_scrap"
        # Get Abstract
        try:
            abstract_html = driver.find_elements_by_css_selector("div[class='abstract author']")
            abstract = abstract_html[0].text
            abstract = re.sub("Abstract\n","",abstract)
            abstract = re.sub("\n","",abstract).strip()
        except:
            abstract = "failed_to_scrap"

        
        attribute = [year, title, conf_jour_name, authors, keywords, abstract]
        if all(a == "failed_to_scrap" for a in attribute): # We stop if we cathed as bot
            break
        else: # Append to the list
            list_source.append("ScienceDirect")
            list_querystring.append(sciencedirect["Query"][index])
            list_querylinks.append(sciencedirect["Query Links"][index])
            list_paperlinks.append(sciencedirect["Paper Links"][index])
            list_year.append(year)
            list_title.append(title)
            list_conf_jour.append(conf_jour_name)
            list_authors.append(authors)
            list_keywords.append(keywords)
            list_abstract.append(abstract)
    except:
        break
driver.quit()

In [28]:
# Wrap to dataframe and safe to excel
sd_metadata = pd.DataFrame({'Source': list_source,
                                  'Query': list_querystring,
                                  'Query Links': list_querylinks,
                                  'Paper Links': list_paperlinks,
                                  'Year':list_year,
                                  'Title':list_title,
                                  'Conf/Journal Name':list_conf_jour,
                                  'Authors':list_authors,
                                  'Keywords':list_keywords,
                                  'Abstract':list_abstract})
sd_metadata.to_excel("./paper-metadata/PaperMetadata_ScienceDirect.xlsx",index=False)